In [ ]:
import os, sys
from pathlib import Path
root_path = Path.cwd().parents[1]
if str(root_path) not in sys.path:
  sys.path.append(str(root_path))
print(f"{root_path=}")

In [ ]:
from katacv.yolov5.parser import get_args_and_writer
from katacv.yolov5.model import get_state
from katacv.utils.model_weights import load_weights

args = get_args_and_writer(no_writer=True, input_args="")
args.path_logs = root_path / "logs"
args.path_cp = args.path_logs / "YOLOv5-checkpoints"
args.load_id = 43
state_ok = load_weights(get_state(args, use_init=False), args)
args.load_id = 42
state_bad = load_weights(get_state(args, use_init=False), args)

In [ ]:
from PIL import Image
import numpy as np
img = []
for p in Path("/home/wty/Pictures/model_test/test_image/8examples").glob('*.jpg'):
  img.append(np.array(Image.open(str(p)).resize((640,640)).convert("RGB")))
img = np.array(img)
print(f"{len(img)=}, {img.dtype=}")

In [ ]:
%load_ext autoreload
%autoreload 2
from katacv.yolov5.predict import Predictor
from katacv.utils.yolo.utils import show_box
predict_ok = Predictor(args, state_ok)
predict_bad = Predictor(args, state_bad)

In [ ]:
for i in range(len(img)):
  x = img[i:i+1].astype(np.float32) / 255.
  pbox = predict_bad.update(x, nms_iou=0.65, nms_conf=0.01)
  show_box(x[0], pbox[0])

In [ ]:
from katacv.utils.yolo.build_dataset import DatasetBuilder
args.path_dataset = Path("/home/wty/Coding/datasets/coco")
args.batch_size = 1
ds_builder = DatasetBuilder(args)
train_ds = ds_builder.get_dataset(subset='train', use_cache=False)
val_ds = ds_builder.get_dataset(subset='val', use_cache=False)

In [ ]:
iter_ds = iter(train_ds)
x = next(iter_ds)[0].numpy().astype(np.float32) / 255.
pbox = predict_ok.update(x, nms_conf=0.01)
show_box(x[0], pbox[0])
pbox = predict_bad.update(x, nms_conf=0.01)
show_box(x[0], pbox[0])

In [ ]:
from katacv.utils.related_pkgs.jax_flax_optax_orbax import *
from katacv.yolov5.loss import cell2pixel
@jax.jit
def predict(state: train_state.TrainState, x: jnp.ndarray):
  logits = state.apply_fn(
    {'params': state.params, 'batch_stats': state.batch_stats},
    x, train=False
  )
  y, batch_size = [], x.shape[0]
  for i in range(3):
    xy = (jax.nn.sigmoid(logits[i][...,:2]) - 0.5) * 2.0 + 0.5
    xy = cell2pixel(xy, scale=2**(i+3))
    wh = (jax.nn.sigmoid(logits[i][...,2:4]) * 2) ** 2 * args.anchors[i].reshape(1,3,1,1,2)
    conf = jax.nn.sigmoid(logits[i][...,4:5])
    cls = jax.nn.sigmoid(logits[i][...,5:])
    conf = conf * jnp.max(cls, axis=-1, keepdims=True)
    cls = jnp.argmax(cls, axis=-1, keepdims=True)
    y.append(jnp.concatenate([xy,wh,conf,cls], -1).reshape(batch_size,-1,6))
  y = jnp.concatenate(y, 1)  # shape=(batch_size,all_pbox_num,6)
  return y
p_ok = jax.device_get(predict(state_ok, x))
p_bad = jax.device_get(predict(state_bad, x))

In [ ]:
np.sort(p_bad[0][:,4])[::-1]

In [ ]:
np.sort(p_ok[0][:,4])[::-1]

In [ ]:
pbox = predict_ok.update(x)
print(pbox)

In [ ]:
show_box(x[0], pbox[0])

In [ ]:
pbox[0]